In [71]:
import gym
import random
import numpy as np
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
import time
from gym.envs.registration import register
from IPython.display import clear_output
from collections import deque

In [72]:
# https://github.com/openai/gym/blob/master/gym/envs/toy_text/frozen_lake.py
# https://github.com/openai/gym/blob/master/gym/envs/__init__.py
try:
    register(
        id='FrozenLakeNoSlip-v0',
        entry_point='gym.envs.toy_text:FrozenLakeEnv',
        kwargs={'map_name' : '4x4', 'is_slippery':False},
        max_episode_steps=100,
        reward_threshold=0.78, # optimum = .8196
    )
except:
    pass

# env_name = "CartPole-v1"
# env_name = "MountainCar-v0"
# env_name = "MountainCarContinuous-v0"
# env_name = "Acrobot-v1"
# env_name = "Pendulum-v0"
env_name = "FrozenLake-v0"
env_name = "FrozenLakeNoSlip-v0"
env = gym.make(env_name)
print("Observation space:", env.observation_space)
print("Action space:", env.action_space)
type(env.action_space)

Observation space: Discrete(16)
Action space: Discrete(4)


gym.spaces.discrete.Discrete

In [73]:
class Agent():
    def __init__(self, env):
        self.is_discrete = \
            type(env.action_space) == gym.spaces.discrete.Discrete
        
        if self.is_discrete:
            self.action_size = env.action_space.n
            print("Action size:", self.action_size)
        else:
            self.action_low = env.action_space.low
            self.action_high = env.action_space.high
            self.action_shape = env.action_space.shape
            print("Action range:", self.action_low, self.action_high)
        
    def get_action(self, state):
        if self.is_discrete:
            action = random.choice(range(self.action_size))
        else:
            action = np.random.uniform(self.action_low,
                                       self.action_high,
                                       self.action_shape)
        return action

In [74]:
class QNAgent(Agent):
    def __init__(self, env, discount_rate=0.97, learning_rate=0.001):
        super().__init__(env)
        self.state_size = env.observation_space.n
        print("State size:", self.state_size)
        
        self.eps = 1.0
        self.discount_rate = discount_rate
        self.learning_rate = learning_rate
        self.exp_buff = deque(maxlen=1000)
        self.batch_size=50
        self.build_model()
        
        self.sess = tf.Session()
        self.sess.run(tf.global_variables_initializer())
        
    def build_model(self):
        tf.reset_default_graph()
        self.state_in = tf.placeholder(tf.int32, shape=[None])
        self.action_in = tf.placeholder(tf.int32, shape=[None])
        self.target_in = tf.placeholder(tf.float32, shape=[None])
        
        self.state = tf.one_hot(self.state_in, depth=self.state_size)
        self.action = tf.one_hot(self.action_in, depth=self.action_size)
        
        self.q_state = tf.layers.dense(self.state, units=self.action_size, name="q_table")
        self.q_action = tf.reduce_sum(tf.multiply(self.q_state, self.action), axis=1)
        
        self.loss = tf.reduce_sum(tf.square(self.target_in - self.q_action))
        self.optimizer = tf.train.AdamOptimizer(self.learning_rate).minimize(self.loss)
        
    def get_action(self, state):
        q_state = self.sess.run(self.q_state, feed_dict={self.state_in: [state]})
        action_greedy = np.argmax(q_state)
        action_random = super().get_action(state)
        return action_random if random.random() < self.eps else action_greedy
    
    def train(self, experience):
        self.exp_buff.append(experience)
        batch = random.choices(self.exp_buff,k=self.batch_size)
        
        
        state, action, next_state, reward, done = (list(col) for col in zip(*batch))
        q_next = self.sess.run(self.q_state, feed_dict={self.state_in: next_state})
        q_next[done] = np.zeros([self.action_size])
        q_target = reward + self.discount_rate * np.max(q_next,axis=1)
        
        feed = {self.state_in: state, self.action_in: action, self.target_in: q_target}
        self.sess.run(self.optimizer, feed_dict=feed)
        
        if experience[4]:
            self.eps = self.eps * 0.99
            
    def __del__(self):
        self.sess.close()
        

In [82]:
agent = QNAgent(env)

Action size: 4
State size: 16


In [84]:
total_reward = 0
wins=0
for ep in range(100):
    state = env.reset()
    done = False
    while not done:
        action = agent.get_action(state)
        next_state, reward, done, info = env.step(action)
        if state==next_state or (done and next_state!=15):
            reward=-1
        agent.train((state,action,next_state,reward,done))
        state = next_state
        total_reward += reward
        if reward>0:
            wins+=1
        print("s:", state, "a:", action)
        print("Episode: {}, Total reward: {}, ,wins: {}, eps: {}".format(ep,total_reward,wins,agent.eps))
        env.render()
        with tf.variable_scope("q_table", reuse=True):
            weights = agent.sess.run(tf.get_variable("kernel"))
            print(weights)
        clear_output(wait=True)

s: 5 a: 1
Episode: 99, Total reward: -260.0, ,wins: 1, eps: 0.13397967485796175
  (Down)
SFFF
FHFH
FFFH
HFFG
[[-0.61576754  0.2273445   0.45306036 -0.43341982]
 [ 0.38573018 -0.677799    0.44651023 -0.42269477]
 [ 0.384981    0.31279474  0.44785172 -0.43496224]
 [ 0.38447383 -0.55494773 -0.50935173 -0.3136669 ]
 [-0.6619221   0.32606745 -0.7056108   0.31491324]
 [-0.40516388  0.17850178 -0.09581068  0.1023919 ]
 [-0.6308132   0.3196562  -0.5443676   0.55926645]
 [ 0.19786787 -0.5352878  -0.08842847  0.48530674]
 [-0.21531296 -0.68524176  0.45200965  0.3182941 ]
 [ 0.25876242  0.06262603  0.45039108 -0.4402708 ]
 [ 0.26125512  0.29234105 -0.4982435   0.5592091 ]
 [-0.04854581 -0.48210627 -0.02442855 -0.5451586 ]
 [ 0.41258305  0.0158934  -0.01303655  0.0305236 ]
 [-0.6196262  -0.18788745  0.44288582  0.51648366]
 [ 0.3766603  -0.56177545  0.06570479  0.40184766]
 [-0.45718658 -0.18561539  0.4787947   0.09715325]]
